### Importing the necessary modules for the project

In [2]:
import numpy as np
from PIL import Image
import cv2
import os

### Displaying the frame from the camera in BGR and GRAY format

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Face Detection

In [4]:
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

1. Creating an object of harrcascade classifier

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(20,20))
    
    for x,y,w,h in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_img = img[y:y+h, x:x+h]
    
    cv2.imshow('Video', img)
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()    

### Creating Training Dataset

In [6]:
users_label_db = {}
face_id = 0

In [8]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

user_name = input('Enter Name of the user : ')
face_id += 1

if face_id not in users_label_db:
    users_label_db[face_id] = user_name
    
print('\n [INFO] Intializing Face Capture. Look at the camera and wait...')

face_count = 0

while True:
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(20,20))
    
    for x,y,w,h in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        face_count += 1
        roi_gray = gray[y:y+h, x:x+w]
        cv2.imwrite(os.path.join('Datasets', 'User.') + str(face_id) + '.' + str(face_count) + '.jpg', roi_gray)
        cv2.imshow('Image', img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27 :
        break
    elif face_count >= 30:
        break
        
print('\n [INFO] Exiting Program and cleanup stuff')

cap.release()
cv2.destroyAllWindows()

Enter Name of the user : Vikas

 [INFO] Intializing Face Capture. Look at the camera and wait...

 [INFO] Exiting Program and cleanup stuff


#### Displaying face_id and face_labels of users

In [9]:
for name,face_id in users_label_db.items():
    print(name, face_id)

1 Vikas
2 Vikas


### Training Recognizer

In [ ]:
path_for_dataset = 'Datasets'

In [ ]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [ ]:
print('\n [INFO] Training Faces to the Face Recognizer')

image_paths = [os.path.join(path_for_dataset, file) for file in os.listdir(path_for_dataset)]

face_samples = []
face_ids = []

for image_path in image_paths:
    
    PIL_img = Image.open(image_path).convert('L')
    img_numpy = np.array(PIL_img, 'uint8')
    
    face_id = int(os.path.split(image_path)[-1].split('.')[1])
    faces = face_detector.detectMultiScale(img_numpy)
    
    for x,y,w,h in faces:
        face_samples.append(img_numpy[y:y+h, x:x+w])
        face_ids.append(face_id)

face_recognizer.train(face_samples, np.array(face_ids))
face_recognizer.write(os.path.join('trainer', 'face_trainer.yml'))

print('\n [INFO] {0} Faces Trained.'.format(len(np.unique(face_ids))))

### Recognizing Faces

In [ ]:
face_recoginzer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read(os.path.join('trainer', 'face_trainer.yml'))
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
text_font = cv2.FONT_HERSHEY_SIMPLEX

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

minW = int(0.1 * cap.get(3))
minH = int(0.1 * cap.get(4))

while True:
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(minW, minH))
    
    for x,y,w,h in faces:
        
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        face_id, confidence = face_recognizer.predict(roi_gray)
        
        if confidence < 100:
            user_name = users_label_db[face_id]
        else:
            user_name = 'Unknown'
        
        confidence = " {0}%".format(100 - confidence)
        
        cv2.putText(img, user_name, (x+5, y-5), text_font, 1, (255,255,255), 2)
        cv2.putText(img, confidence, (x+5, y+h-5), text_font, 1, (255,255,255), 2)
        
    cv2.imshow('Video', img)
    
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break
    
print('\n [INFO] Exiting program and cleanup stuff')
cap.release()
cv2.destroyAllWindows()